In [1]:
import pathlib
import pandas as pd
import asyncio
from aiohttp import ClientSession
import requests
import base64
import api_setup
import spotipy
REPO_ROOT = pathlib.Path.cwd().parent

In [2]:
# API Auth
env_vars = api_setup.parse_api_kvs(pathlib.Path.cwd().parent / "api-keys")
auth_manager = spotipy.SpotifyClientCredentials(env_vars['client_id'], env_vars['client_secret'])
spotify = spotipy.Spotify(client_credentials_manager=auth_manager, backoff_factor=2)

In [3]:
from typing import List

In [4]:
def get_songs_from_playlist(spotify_client: spotipy.Spotify, playlist_uri: str) -> List[str]:
    """
    Return a list of strings of the URIs of the tracks in this playlist.
    """
    tracks_json = spotify_client.playlist_items(playlist_uri)
    return [track['track']['uri'][14:] for track in tracks_json['items']]

get_songs_from_playlist(spotify, "spotify:playlist:76S2ElS2cyzY624wGBGKpB")

['7yl0ItOwlAnALSctbUiavO',
 '4LpUpiYoZ2M3Z1kmhn4EQo',
 '1OuN92HcVG6NVpWbeESNB3',
 '44SO1hMPfH9xUvmI7bjhou',
 '58e7V70Em6FABOiln4jNoZ',
 '2Xl2dfsBQYaPP5I2viTVr9',
 '7L4G39PVgMfaeHRyi1ML7y',
 '1GpZofCtuWj4adPQLqpeFw',
 '3q6ygCZID0OKj6MUxInB48']

In [9]:
# async api calls
from pprint import PrettyPrinter
pp = PrettyPrinter()

async def get_audio_features(session: ClientSession, track_uri: str) -> dict:
    """
    Return the audio features of the song with the given uri.
    """
    uri = track_uri.split(":")[-1]
    endpoint = f"https://api.spotify.com/v1/audio-features/{uri}"

    async with session.get(endpoint) as response:
        response = await(response.json())
    return response

async def get_artist(session: ClientSession, artist_uri: str) -> dict:
    """
    Given an artist's URI, return their info.
    """
    uri = artist_uri.split(":")[-1]
    endpoint = f"https://api.spotify.com/v1/artists/{uri}"
    
    async with session.get(endpoint) as response:
        response = await(response.json())
    
    return response

async def get_artist_from_track_uri(session: ClientSession, track_uri: str) -> dict:
    """
    Given a track URI, return its artist's name and their popularity.
    """
    uri = track_uri.split(":")[-1]
    endpoint = f"https://api.spotify.com/v1/tracks/{uri}"
    
    async with session.get(endpoint) as response:
        response = await(response.json())
        artist_uri = response['artists'][0]['uri']
        artist_info = await(get_artist(session, artist_uri))
    print(response)
    artist_name, artist_popularity, artist_genres = artist_info['name'], artist_info['popularity'], artist_info['genres']
    return {'track_uri': uri, 'artist_name': artist_name, 'artist_popularity': artist_popularity, 'artist_genres': artist_genres}

def get_header_with_token(client_id: str, client_secret: str):
    creds = f"{env_vars['client_id']}:{env_vars['client_secret']}"
    creds_b64 = base64.b64encode(creds.encode())
    headers= {"Authorization": f"Basic {creds_b64.decode()}"}
    data= {"grant_type": "client_credentials"}
    token = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
    token = token.json()['access_token']
    return {"Accept": "application/json", "Content-Type": "application/json", "Authorization": f"Bearer {token}"}

async def featurize_song_list(client_id:str, client_secret: str, song_uris: List[str]) -> List[dict]:
    # TODO: This can be chunked >.>
    # https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features
    request_headers = get_header_with_token(client_id, client_secret)
    async with ClientSession(headers=request_headers) as session:
        tasks = [asyncio.ensure_future(get_audio_features(session, uri)) for uri in song_uris]
        features = await(asyncio.gather(*tasks))
    return features

async def get_playlist_song_features(spotify_client: spotipy.Spotify, client_id:str, client_secret: str, playlist_uri: str) -> dict:
    song_uris = get_songs_from_playlist(spotify_client, playlist_uri)
    playlist_song_features = await(featurize_song_list(client_id, client_secret, song_uris))
    return playlist_song_features

async def dataframe_from_playlist(spotify_client: spotipy.Spotify, client_id:str, client_secret: str, playlist_uri: str) -> pd.DataFrame:
    # Get the song features and URIs from a playlist
    playlist_song_features = await(get_playlist_song_features(spotify_client, client_id, client_secret, playlist_uri))
    playlist_song_uris = [features['uri'] for features in playlist_song_features]
    # Get those songs' artists and their popularity
    async with ClientSession(headers=get_header_with_token(client_id, client_secret)) as session:
        tasks = [asyncio.ensure_future(get_artist_from_track_uri(session, uri)) for uri in playlist_song_uris]
        artist_info = await(asyncio.gather(*tasks))
    
    # Create the dataframe we expect >:(
    song_features_df = pd.DataFrame.from_records(playlist_song_features)
    track_uri_artist_popularity_df = pd.DataFrame.from_records(artist_info).set_index('track_uri')
    
    song_features_df = song_features_df.join(track_uri_artist_popularity_df, on='uri')
    
    return song_features_df
    
    
    

args = (spotify, env_vars['client_id'], env_vars['client_secret'], "spotify:playlist:76S2ElS2cyzY624wGBGKpB")
t = await(dataframe_from_playlist(*args))

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1A9o3Ljt67pFZ89YtPPL5X'}, 'href': 'https://api.spotify.com/v1/artists/1A9o3Ljt67pFZ89YtPPL5X', 'id': '1A9o3Ljt67pFZ89YtPPL5X', 'name': 'Snoh Aalegra', 'type': 'artist', 'uri': 'spotify:artist:1A9o3Ljt67pFZ89YtPPL5X'}], 'available_markets': [], 'external_urls': {'spotify': 'https://open.spotify.com/album/42wtqDcTQlJJbUzAPBSwaK'}, 'href': 'https://api.spotify.com/v1/albums/42wtqDcTQlJJbUzAPBSwaK', 'id': '42wtqDcTQlJJbUzAPBSwaK', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27353c8eea457ff7ca8fbe32d68', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e0253c8eea457ff7ca8fbe32d68', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d0000485153c8eea457ff7ca8fbe32d68', 'width': 64}], 'name': '- Ugh, those feels again', 'release_date': '2019-08-16', 'release_date_precision': 'day', 'total_tracks': 14, 'type': 'albu

In [8]:
t

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_name,artist_popularity,artist_genres
0,0.411,0.134,11,-17.251,0,0.0309,0.9760,0.939000,0.0919,0.0658,...,audio_features,7yl0ItOwlAnALSctbUiavO,spotify:track:7yl0ItOwlAnALSctbUiavO,https://api.spotify.com/v1/tracks/7yl0ItOwlAnA...,https://api.spotify.com/v1/audio-analysis/7yl0...,199708,3,NaN,NaN,NaN
1,0.709,0.496,1,-10.528,0,0.0417,0.5810,0.191000,0.1510,0.4690,...,audio_features,4LpUpiYoZ2M3Z1kmhn4EQo,spotify:track:4LpUpiYoZ2M3Z1kmhn4EQo,https://api.spotify.com/v1/tracks/4LpUpiYoZ2M3...,https://api.spotify.com/v1/audio-analysis/4LpU...,181587,4,NaN,NaN,NaN
2,0.875,0.599,9,-9.077,0,0.1380,0.0333,0.014300,0.1130,0.1120,...,audio_features,1OuN92HcVG6NVpWbeESNB3,spotify:track:1OuN92HcVG6NVpWbeESNB3,https://api.spotify.com/v1/tracks/1OuN92HcVG6N...,https://api.spotify.com/v1/audio-analysis/1OuN...,256507,4,NaN,NaN,NaN
3,0.692,0.637,10,-6.449,1,0.1500,0.2860,0.000022,0.2310,0.4290,...,audio_features,44SO1hMPfH9xUvmI7bjhou,spotify:track:44SO1hMPfH9xUvmI7bjhou,https://api.spotify.com/v1/tracks/44SO1hMPfH9x...,https://api.spotify.com/v1/audio-analysis/44SO...,222067,4,NaN,NaN,NaN
4,0.528,0.740,0,-8.211,0,0.0347,0.4180,0.232000,0.3180,0.3420,...,audio_features,58e7V70Em6FABOiln4jNoZ,spotify:track:58e7V70Em6FABOiln4jNoZ,https://api.spotify.com/v1/tracks/58e7V70Em6FA...,https://api.spotify.com/v1/audio-analysis/58e7...,340880,4,NaN,NaN,NaN
5,0.743,0.676,5,-10.249,1,0.2020,0.2500,0.000000,0.1410,0.8590,...,audio_features,2Xl2dfsBQYaPP5I2viTVr9,spotify:track:2Xl2dfsBQYaPP5I2viTVr9,https://api.spotify.com/v1/tracks/2Xl2dfsBQYaP...,https://api.spotify.com/v1/audio-analysis/2Xl2...,204480,4,NaN,NaN,NaN
6,0.463,0.273,0,-15.364,0,0.0740,0.9070,0.000367,0.1010,0.2930,...,audio_features,7L4G39PVgMfaeHRyi1ML7y,spotify:track:7L4G39PVgMfaeHRyi1ML7y,https://api.spotify.com/v1/tracks/7L4G39PVgMfa...,https://api.spotify.com/v1/audio-analysis/7L4G...,239960,4,NaN,NaN,NaN
7,0.389,0.740,6,-11.740,0,0.4790,0.7110,0.000227,0.3390,0.6000,...,audio_features,1GpZofCtuWj4adPQLqpeFw,spotify:track:1GpZofCtuWj4adPQLqpeFw,https://api.spotify.com/v1/tracks/1GpZofCtuWj4...,https://api.spotify.com/v1/audio-analysis/1GpZ...,157637,4,NaN,NaN,NaN
8,0.713,0.755,7,-4.577,1,0.1510,0.2620,0.000114,0.0947,0.6250,...,audio_features,3q6ygCZID0OKj6MUxInB48,spotify:track:3q6ygCZID0OKj6MUxInB48,https://api.spotify.com/v1/tracks/3q6ygCZID0OK...,https://api.spotify.com/v1/audio-analysis/3q6y...,206968,4,NaN,NaN,NaN
